In [99]:
# import libraries
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import os.path
import time
from distanceMatrix import *
# define functions
%run distanceMatrix.py

# instance size
N = 30
# number of trucks
K = 10
# maximum tour time
T = int(600)  # 4.5h = 270min

# load the data and files
clientsFile = 'clients'+str(N)+'.csv'
# 0     =7am
# 600   =5pm
# noon  =300
# 1pm   =360
# 4pm   =540
clients = pd.read_csv(clientsFile)
wpfsFile = 'WPF.csv'
citiesFile = 'belgian-cities-geocoded.csv'
cities = pd.read_csv(citiesFile)
distanceMatrixFile = 'distanceMatrix'+str(N)+'.csv'
wpfs = pd.read_csv(wpfsFile)

# get the matrix and save it if necessary
if os.path.isfile(distanceMatrixFile):
    print('exists')
    distanceMatrix = pd.read_csv(distanceMatrixFile).to_numpy()
else:
    print('does not exist')
    t_matrix_start = time.time()
    distanceMatrix = createDistanceMatrix(clientsFile, citiesFile, wpfsFile)
    t_matrix_end = time.time()
    pd.DataFrame(distanceMatrix).to_csv(distanceMatrixFile, index=False)
    t_matrix = t_matrix_end-t_matrix_start
    print(t_matrix)

# get coordinates
coords = [(50.9338827, 4.5605498)]  # depot
for client in clients['Place']:
    idx = cities.index[cities['name'] == client].tolist()[0]
    coords.append((cities.iloc[idx]['lat'], cities.iloc[idx]['lng']))

# get timewindows
timeWindows = [(0, 600)]  # depot
for id in clients['ClientID']:
    opening = clients['opening'][id-1]
    closing = clients['closing'][id-1]
    opening = int(opening)
    closing = int(closing)
    timeWindow = (opening, closing)
    timeWindows.append(timeWindow)


exists


In [100]:
# build the model and solve it
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # data['time_matrix'] = [
    #     [0,7.07,7.07,7.07,7.07],
    #     [7.07,0,10,2*7.07,10],
    #     [7.07,10,0,10,2*7.07],
    #     [7.07,2*7.07,10,0,10],
    #     [7.07,10,2*7.07,10,0]
    # ]
    data['time_matrix'] = distanceMatrix
    data['time_windows'] = timeWindows
    # data['time_windows'] = [
    #     (0, 600),  # depot
    #     (0, 600),  # 1
    #     (0, 600),  # 2
    #     (0, 600),  # 3
    #     (0, 600),  # 4
    #     (0, 600)
    # ]
    data['num_vehicles'] = K
    data['depot'] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))

def get_cumul_data(solution, routing, dimension):
  """Get cumulative data from a dimension and store it in an array."""
  # Returns an array cumul_data whose i,j entry contains the minimum and
  # maximum of CumulVar for the dimension at the jth node on route :
  # - cumul_data[i][j][0] is the minimum.
  # - cumul_data[i][j][1] is the maximum.

  cumul_data = []
  for route_nbr in range(routing.vehicles()):
    route_data = []
    index = routing.Start(route_nbr)
    dim_var = dimension.CumulVar(index)
    route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      dim_var = dimension.CumulVar(index)
      route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    cumul_data.append(route_data)
  return cumul_data

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

########################################
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Time Windows constraint.
time = 'Time'
routing.AddDimension(
    transit_callback_index,
    0,  # allow waiting time
    T,  # maximum time per vehicle
    False,  # Don't force start cumul to zero.
    time)
time_dimension = routing.GetDimensionOrDie(time)
# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
# Add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])

# Instantiate route start and end times to produce feasible times.
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
# maximum computation time
search_parameters.time_limit.seconds = 30

# search_parameters = pywrapcp.DefaultRoutingSearchParameters()
# search_parameters.local_search_metaheuristic = (
#     routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC)
# search_parameters.time_limit.seconds = 60
# search_parameters.log_search = True


# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
    # get solution windows
    solution_windows=get_cumul_data(solution,routing,time_dimension)
    solution_routes=get_routes(solution,routing,manager)
    
else:
    print("INFEASIBLE or RAN OUT OF COMPUTATION TIME")  

Objective: 4642
Route for vehicle 0:
0 Time(0,0) -> 16 Time(262,262) -> 8 Time(492,492) -> 0 Time(558,558)
Time of the route: 558min

Route for vehicle 1:
0 Time(0,0) -> 1 Time(137,137) -> 17 Time(357,357) -> 23 Time(534,534) -> 0 Time(557,557)
Time of the route: 557min

Route for vehicle 2:
0 Time(0,0) -> 19 Time(206,206) -> 11 Time(398,398) -> 0 Time(450,450)
Time of the route: 450min

Route for vehicle 3:
0 Time(0,0) -> 29 Time(173,173) -> 28 Time(326,326) -> 13 Time(498,498) -> 12 Time(566,566) -> 0 Time(588,588)
Time of the route: 588min

Route for vehicle 4:
0 Time(0,0) -> 26 Time(121,121) -> 4 Time(268,268) -> 0 Time(308,308)
Time of the route: 308min

Route for vehicle 5:
0 Time(0,0) -> 3 Time(44,44) -> 27 Time(162,162) -> 22 Time(280,280) -> 9 Time(455,455) -> 0 Time(598,598)
Time of the route: 598min

Route for vehicle 6:
0 Time(0,0) -> 24 Time(72,72) -> 21 Time(233,233) -> 6 Time(302,302) -> 15 Time(393,393) -> 0 Time(541,541)
Time of the route: 541min

Route for vehicle 7:


In [115]:
# Map the solution
import folium

# list of colors to be used in the map
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred','black', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray','red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# zoom in on Belgium
map = folium.Map(location=[50.5,4], zoom_start = 8)

# route 0 example
postProcessedRoutes = [
    [ # route 0
        (50.9338827, 4.5605498),#depot
        (50.98707, 5.36716),#zonhoven
        (51.0258791, 4.4775365),#mechelen
        (50.98707, 5.36716),#zonhoven
        (50.9215166, 5.3447405),#hasselt
        (51.2198771, 4.4011356),#antwerp
        (50.9215166, 5.3447405),#hasselt
        (51.2198771, 4.4011356),#antwerp
        (50.9215166, 5.3447405),#hasselt
        (50.9338827, 4.5605498),#depot
    ],
    [], # route 1
    [ # route 2
        (50.9338827, 4.5605498),# depot
        (50.8492265, 2.8779465),# ieper
        (51.2093488, 3.2247013),# brugge
        (50.8194894, 3.2577076),# kortjrijk
        (51.0678307, 3.7290914),# gent
        (50.8427501, 4.3515499),# brussel
        (51.0258791, 4.4775365),# mechelen
        (51.0953745, 4.5052038),# duffel
        (50.9338827, 4.5605498),# depot
    ]
]

for idx,postProcessedRoute in enumerate(postProcessedRoutes):
    if len(postProcessedRoute)>0:
        folium.PolyLine(postProcessedRoute,weight=len(postProcessedRoute),color=colors[idx],popup='route'+str(idx)).add_to(map)
        


# visualize the routes
# solution_routes_nonEmpty = [route for route in solution_routes if len(route) > 2]
# for idx,route in enumerate(solution_routes_nonEmpty):
#     route_coords=[]
#     for node in route:
#         route_coords.append(coords[node])
#     folium.PolyLine(route_coords,weight=len(route),color=colors[idx],popup=route).add_to(map)

    

# visualize the nodes
for idx,coord in enumerate(coords):
    if(idx == 0):
        folium.Marker(
            location=[coord[0], coord[1]],
            popup='#0\nKampenhout',
            icon=folium.Icon(color='black',icon="home"),
        ).add_to(map)
    else:
        icon_colors = ['green','lightblue','cadetblue','orange','lightgray','red']
        folium.Marker(
            location=[coord[0], coord[1]],
            popup='#\n'+str(clients['ClientID'][idx-1])+' '+clients['Place'][idx-1]+'\n'+str(clients['ActionType'][idx-1])+'\n'+str(coord),
            icon=folium.Icon(color=icon_colors[clients['ActionType'][idx-1]-1], icon='', prefix='fa')
        ).add_to(map)

# visualize the WPFs
for idx,wpf in enumerate(wpfs['Place']):
    folium.Marker(
        location=[wpfs.iloc[idx].lat, wpfs.iloc[idx].lon],
        popup=wpfs.iloc[idx].Place+'\n'+str(wpfs.iloc[idx].lat)+', '+str(wpfs.iloc[idx].lon),
        icon=folium.Icon(color='black',icon="trash",prefix='fa'),
    ).add_to(map)

map.save('routing'+str(N)+'.html')
map


In [102]:
a=0
b=4
distanceKmFromCoord(
    # 50.98707, 5.36716, #zonhoven
    # 50.9215166, 5.3447405, #hasselt
    # 50.9338827, 4.5605498, #depot - kampenhout
    # 51.2198771, 4.4011356, # antwerp
    51.0258791, 4.4775365, # mechelen
    # 51.191087, 5.1170647, # mol
    # 50.9806151, 4.827381 # aarschot
    # 51.0731945, 2.6680059, # veurne
    # 51.2093488, 3.2247013, # brugge
    # 51.08608, 4.36633, #boom
    # 50.8492265, 2.8779465, #ieper
    # 50.8194894, 3.2577076, #kortjrijk
    # 51.0678307, 3.7290914, #gent
    # 50.8427501, 4.3515499, #brussel
    # 51.0953745, 4.5052038, # duffel
    # 50.693646, 5.5638281, # liers
    # 51.0010401, 5.0852169, # dies
    # 51.1987114, 3.8119772, # zelzate
    # 50.668081, 4.6118324, # louvain
    # 51.16257, 4.99084, # geel
    # 50.81057, 4.93622, # tienen
    # 50.8815197, 4.6967578, #leuven
    # 50.64786, 5.54357, # glein
    # 50.84109, 4.83471, # boutersem
    # 51.1768785, 4.835648, # herentals
    # 51.0614235, 4.5055555, # sint-katelijne
    # 50.9356235, 4.3785596, #grimbergen
    50.8795024, 4.4826212, #zaventem
    # 51.0678307, 3.7290914, # gent
)/50*60

19.54242453684883

In [103]:
# %run generateClients.py
# generateClients(200,'belgian-cities-geocoded.csv')

In [104]:
# %run distanceMatrix.py
# createDistanceMatrix(clientsFile, citiesFile, wpfsFile)

3:  Antwerpen Grimbergen 37.988179837638135
WPFS: Grimbergen 14.639262386386157 Mechelen Grimbergen
ST: @Grimbergen 73.27852477277231
3:  Antwerpen Leuven 59.87307814979073
ST: @Leuven 25
3:  Antwerpen Mechelen 60.56901107234176
ST: @Mechelen 25
3:  Antwerpen Zaventem 56.229442152742045
ST: @Zaventem 25
3:  Antwerpen Gent 118.53857113982298
ST: @Gent 30
3:  Antwerpen Sint-Katelijne-Waver 22.886189515685892
WPFS: Sint-Katelijne-Waver 5.295036493543709 Mechelen Sint-Katelijne-Waver
ST: @Sint-Katelijne-Waver 54.59007298708741
3:  Antwerpen Hasselt 88.59508011784767
WPFS: Hasselt 88.59508011784767 Antwerpen Hasselt
ST: @Hasselt 221.19016023569534
3:  Antwerpen Glain 137.7540920065553
ST: @Glain 25
3:  Antwerpen Boutersem 72.6441402887803
ST: @Boutersem 30
3:  Antwerpen Boom 72.49881985106474
ST: @Boom 25
3:  Antwerpen Duffel 68.49704612146556
ST: @Duffel 12
3:  Antwerpen Brussel 67.83927468102551
ST: @Brussel 40
3:  Antwerpen Leuven 59.87307814979073
ST: @Leuven 25
3:  Antwerpen Brugge 164

array([[  0., 137.,  88.,  44.,  45.,  41., 108.,  73., 287., 122.,  62.,
         57.,  40.,  67.,  44., 149., 262., 218.,  71., 206.,  52., 108.,
         44., 163.,  72.,  72., 121.,  78., 142., 173.,  67.],
       [ 40.,   0., 111.,  85.,  86.,  81., 149.,  78., 310., 163., 103.,
         97.,  80., 108.,  85., 189., 281., 220., 111., 246.,  93., 149.,
         85., 188.,  91., 112., 162., 119., 182., 214., 107.],
       [ 15., 135.,   0.,  60.,  60.,  56., 123.,  75., 302., 138.,  78.,
         72.,  55.,  83.,  60., 164., 277., 231.,  86., 221.,  68., 123.,
         60., 178.,  87.,  87., 137.,  94., 157., 188.,  82.],
       [ 67., 204., 155.,   0., 112., 108., 140., 139., 354., 175., 116.,
        124.,  68., 113., 111., 179., 329., 285., 108., 273., 119., 137.,
         98., 230., 139., 125., 188., 118., 178., 208., 121.],
       [ 40., 178., 128.,  85.,   0.,  81., 148., 113., 327., 162., 102.,
         57.,  80., 108.,  85., 189., 302., 258., 111., 212.,  93., 148.,
        